In [9]:
#Ejercicio 7


import pandas as pd

import os

import numpy as np

import scipy.io as sio

from scipy import stats

from math import factorial

import seaborn as sns

from scipy.linalg import eigh,eig,eigvals

from scipy.optimize import linprog

import matplotlib.pyplot as plt

from copy import deepcopy

import sys

from scipy.spatial import distance as dist

import copy


# Ejercicio 7: Plan de evacuación de emergencia (usando simplex)

import numpy as np
from scipy.optimize import linprog
from basic_utils import nn2na, get_usage_string, get_min_cut

# Datos
NN = np.array([
    [0,     1, 1, 1, 1, 1, 1,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0],
   
    [0, 0, 0, 0, 0, 0, 0,   0, 1, 0, 0, 0, 0,   0, 0, 0, 1, 0, 0,   0, 0, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 1, 0, 0, 0,   0, 0, 0, 0, 1, 0,   0, 0, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 1, 0, 0,   0, 0, 0, 0, 0, 1,   0, 0, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 1, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 1,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0],
   
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 1, 0, 0, 0,   0, 1, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 1, 0, 0,   0, 0, 1, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 1, 0,   0, 0, 0, 1, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 1,   0, 0, 0, 0, 1, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 1,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0],
   
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 1, 0, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 1, 0, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 1, 0, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 1, 0,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 1,   0],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0],
   
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   1],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   1],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   1],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   1],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   1],
    [0, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   1],
   
    [1, 0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0, 0, 0, 0, 0, 0,   0]
                ])

Aeq, arc_idxs = nn2na(NN)

C = np.zeros((1,Aeq.shape[1]))
C[0,Aeq.shape[1]-1]=-1

beq = np.zeros((1,Aeq.shape[0]))

max_q = [np.inf, np.inf , np.inf, np.inf, np.inf,np.inf,
         5,10,5,10,5,10,5,5,
         6,3,6,3,6,3,6,3,3,
         3,3,3,3,3,
         np.inf, np.inf , np.inf, np.inf, np.inf,np.inf,\
         np.inf]

bounds = tuple([(0, max_q[arcs]) for arcs in range(0, Aeq.shape[1])])

# print('## Optimizer inputs ## \n'
#       'Cost vector: %s \n '
#       'A_eq Node-Arc matrix:\n%s \n'
#       'b_eq demand-supply vector: %s \n'
#       'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

# GET THE SOLUTION:
usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1
print('## Results ## \n'
      'The usage of each arc will be (from, to): %s \n'
      'The arcs that produce the minimum cut (from, to): %s \n'
      "\n"
      "\n"
      'The maximum flow will be: %0.2f ' % (usage, min_cut , max_flow))


## Results ## 
The usage of each arc will be (from, to): {(0, 1): '6/inf', (0, 2): '6/inf', (0, 3): '3/inf', (0, 4): '3/inf', (0, 5): '0/inf', (0, 6): '0/inf', (1, 8): '5/5', (1, 16): '1/10', (2, 9): '5/5', (2, 17): '1/10', (3, 10): '3/5', (3, 18): '0/10', (4, 11): '3/5', (5, 12): '0/5', (7, 15): '0/6', (7, 20): '0/3', (8, 16): '2/6', (8, 21): '3/3', (9, 17): '2/6', (9, 22): '3/3', (10, 18): '0/6', (10, 23): '3/3', (11, 24): '3/3', (13, 20): '0/3', (14, 21): '0/3', (15, 22): '0/3', (16, 23): '3/3', (17, 24): '3/3', (19, 25): '0/inf', (20, 25): '0/inf', (21, 25): '3/inf', (22, 25): '3/inf', (23, 25): '6/inf', (24, 25): '6/inf', (25, 0): '18/inf'} 
The arcs that produce the minimum cut (from, to): [(1, 8), (2, 9), (8, 21), (9, 22), (10, 23), (11, 24), (16, 23), (17, 24)] 


The maximum flow will be: 18.00 


<ipython-input-9-b019be411e7d>:97: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
